In [1]:
import numpy as np
import numpyro
import numpyro.distributions as dist
from jax import random
from numpyro.infer import MCMC, HMC,NUTS
import fitsio
import matplotlib as mpl
from multiprocessing import Pool
from scipy.stats import dirichlet
from astropy.table import Table
from astropy.io import fits
import fitsio
from astropy.cosmology import Planck18
import jax.numpy as jnp
import matplotlib.pyplot as plt
from numpyro import handlers 
from jax import random, jit,vmap
import jax
from fitting import abs_mag_to_L,get_lfpars_shen20,T0,T1,T2,get_lfpars,get_phis
from jax import random, vmap
from jax.scipy.stats import norm
import numpyro.distributions as dist
from jax.scipy.special import gammaln
from astropy.cosmology import WMAP9 as cosmo
from scipy.optimize import curve_fit, brentq
import fitting

In [2]:
k = Table.read('/pscratch/sd/z/zhaozhon/DATASET/DESI-data/k.csv',format='csv')
indice = np.isnan(k['k_gg']) | np.isinf(k['k_gg']) | np.isnan(k['k_rr']) | np.isinf(k['k_rr'])
k_gg = k['k_gg'][~indice]
k_rr = k['k_rr'][~indice]

In [3]:
file = '/pscratch/sd/z/zhaozhon/DATASET/DESI-data/new_d.fits'
d = Table(fitsio.read(file))
SEL = np.where((d['Z'] > 2.3) & (d['Z'] < 2.8))[0]
TARGETID = d['TARGETID'][SEL][~indice]
apr_mag_g = d['APP_MAG_G'][SEL][~indice]
apr_mag_g_err = d['APP_MAGERR_G'][SEL][~indice]
apr_mag_r = d['APP_MAG_R'][SEL][~indice]
apr_mag_r_err = d['APP_MAGERR_R'][SEL][~indice]
z_desi = d['Z'][SEL][~indice]

In [4]:
d_desi = Planck18.distmod(z_desi).value

In [5]:
TARGETID, indices = np.unique(TARGETID, return_index=True)
apr_mag_g = apr_mag_g[indices]
apr_mag_g_err = apr_mag_g_err[indices]
apr_mag_r = apr_mag_r[indices]
apr_mag_r_err = apr_mag_r_err[indices]
d_desi = d_desi[indices]
z_desi = z_desi[indices]
k_gg = k_gg[indices]
k_rr = k_rr[indices]
N_obs = TARGETID.shape[0]
print(N_obs)

157741


In [6]:
# Assuming z_desi is your redshift array
filter_mask = (z_desi > 2.3) & (z_desi < 2.31)

# Apply the filter to all arrays
TARGETID = TARGETID[filter_mask]
#apr_mag_g = apr_mag_g[filter_mask]
#apr_mag_g_err = apr_mag_g_err[filter_mask]
apr_mag_r = apr_mag_r[filter_mask]
apr_mag_r_err = apr_mag_r_err[filter_mask]
d_desi = d_desi[filter_mask]
z_desi = z_desi[filter_mask]
#k_gg = k_gg[filter_mask]
k_rr = k_rr[filter_mask]

# Recalculate N_obs after filtering
N_obs = TARGETID.shape[0]
print(N_obs)

4434


In [7]:
apr_mag_g = apr_mag_g.astype(np.float64)
apr_mag_g_err = apr_mag_g_err.astype(np.float64)
k_gg = k_gg.astype(np.float64)

apr_mag_r = apr_mag_r.astype(np.float64)
apr_mag_r_err = apr_mag_r_err.astype(np.float64)
k_rr = k_rr.astype(np.float64)

d_desi = d_desi.astype(np.float64)
z_desi = z_desi.astype(np.float64)

In [8]:
print(get_phis(abs_mag_to_L(np.linspace(-40,0,100)),np.linspace(2.3,2.8,100),"shen20").max())

0.4783756734127197


In [26]:
N_obs = fitting.N_obs(2.3,2.31)
gamma1, gamma2, L_star, _ = get_lfpars_shen20(2.305)
alpha = -(gamma1+1)
beta = -(gamma2+1)
print(alpha,beta)
L_min = abs_mag_to_L(apr_mag_r.max()-k_rr.mean()-d_desi.mean()-0)/L_star
L_max = abs_mag_to_L(apr_mag_r.min()-k_rr.mean()-d_desi.mean()-0)/L_star
#L_max = 1000000

N = N_obs(21.1,2,1.81,alpha,beta,L_min,L_max,k_rr.mean(),apr_mag_r_err.mean())
print(N)

-1.409894 -3.5141931173975998
0.6774358026315073
0.9528879766404953
845993213.8526562 6.227823720528854e-06 4.483228216492882e+46
8589.681360311013


In [10]:
i_start = 0
i_end = i_start+1000
print(N_obs,0.16*N_pre(m_samples, z_samples, d_samples, k_samples, comoving_volume,-1.6050, 17.1728,0.3039, 7, 100, apr_mag_r[i_start:i_end], z_desi[i_start:i_end], k_rr[i_start:i_end]))
print(jnp.sum(integral(k_rr[i_start:i_end], d_desi[i_start:i_end], -1.8050, apr_mag_r[i_start:i_end], apr_mag_r_err[i_start:i_end], z_desi[i_start:i_end], 18.1728, 0.3039, 11.9,200)) , log_poisson(N_obs, 0.16 * N_pre(m_samples, z_samples, d_samples,k_samples, comoving_volume,-1.8050, 18.1728, 0.3039, 11.9, 100, apr_mag_r[i_start:i_end], z_desi[i_start:i_end], k_rr[i_start:i_end])))

NameError: name 'N_pre' is not defined

In [ ]:
rng_key = random.PRNGKey(0)
# Sampler configuration
num_warmup, num_samples = 15000, 10000
i_start = 0
i_end = i_start+1000
# Initialize batch parameters for the entire datasetw

init_params = {
    'x': -1.85 * jnp.ones((1,)),  
    'm0': 18.69 * jnp.ones((1,)),
    'a': 0.2 * jnp.ones((1,)),
    'b': 12 * jnp.ones((1,)),
    #'f': 0.16 * jnp.ones((1,)),
}

# Set up the NUTS sampler
kernel = NUTS(model, target_accept_prob=0.8)
#kernel = HMC(model) 

# Run the sampler over thswwe entire dataset
mcmc = MCMC(kernel, num_warmup=num_warmup, num_samples=num_samples, num_chains=1, jit_model_args=True, progress_bar=True)
#mcmc.run(rng_key, apr_mag_g, apr_mag_g_err, k_gg, apr_mag_r, apr_mag_r_err, k_rr, d,z, 22.5, init_params=init_params)
mcmc.run(rng_key,m_samples, z_samples, d_samples,k_samples, comoving_volume, k_rr[i_start:i_end], d_desi[i_start:i_end], apr_mag_r[i_start:i_end], apr_mag_r_err[i_start:i_end], z_desi[i_start:i_end],N_obs)
# Get samples
samples = mcmc.get_samples()
# Optionally, process or save your samples
print(samples)

In [ ]:
# Extract the chains and parameters
x = samples['x']
m0 = samples['m0']
a = samples['a']
b = samples['b']
#f = samples['f']
#c = samples['c']
# Creating a figure to hold the subplots
fig, axs = plt.subplots(4, 2, figsize=(12, 24))  

# Helper function to plot histograms and traces
def plot_param(ax, data, title):
    ax[0].hist(data, bins=100, alpha=0.75, color='blue')
    ax[0].set_title(f'Posterior Distribution of {title}')
    ax[1].plot(data, color='blue')
    ax[1].set_title(f'Trace of {title}')

# Plotting each parameter
plot_param(axs[0], x, 'x')
plot_param(axs[1], m0, 'm0')
plot_param(axs[2],a,'a')
plot_param(axs[3],b,'b')
#plot_param(axs[4],f,'f')
#plot_param(axs[4],c,'c')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
import corner
# Extract samples for plotting
#samples = np.vstack((x, m0)).T
samples = np.vstack((x,m0,a,b)).T
# Create a corner plot
fig = corner.corner(samples, labels=['x', 'm0','a','b'], 
                    show_titles=True,quantiles=[0.16, 0.5, 0.84],
                       plot_contours=True, color='blue', title_kwargs={"fontsize": 12},title_fmt=".4f")

fig.suptitle('Corner Plot of Parameters')
fig.show()